<a href="https://colab.research.google.com/github/GPT-Design/SCIENCE/blob/main/lenstronomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Mount Google Drive, Load FITS file (Hubble Frontier Fields data)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to working directory and list FITS files
import os, glob
from astropy.io import fits
import matplotlib.pyplot as plt

os.chdir('/content/drive/My Drive/AI/3TE/LENSING')

fits_files = glob.glob('*.fits')
print("Available FITS files:", fits_files)

# Load first FITS file (as example, replace with loop if desired)
fits_image_path = fits_files[0]

with fits.open(fits_image_path) as hdul:
    image_data = hdul[0].data

plt.imshow(image_data, origin='lower', cmap='inferno', vmax=image_data.mean() + 3*image_data.std())
plt.colorbar(label='Pixel intensity')
plt.title('HFF Lensing Image')
plt.show()


 2. lenstronomy Setup

In [ ]:
# Ensure lenstronomy is installed:
!pip install lenstronomy==1.12.5

3. Define lens model and parameters
Here's a basic example using a Singular Isothermal Ellipsoid (SIE). Replace or extend this model according to the specifics of your 3T_E parameters.

In [ ]:
from lenstronomy.LensModel.lens_model import LensModel
import numpy as np

# Singular Isothermal Ellipsoid (SIE) lens model
lens_model_list = ['SIE']

# Example parameters, adjust as necessary for your data
kwargs_lens = [{
    'theta_E': 1.2,  # Einstein radius (arcsec)
    'e1': 0.05,
    'e2': -0.05,
    'center_x': 0.0,
    'center_y': 0.0
}]

# Instantiate lens model
lens_model = LensModel(lens_model_list=lens_model_list)


4. Data Configuration for lenstronomy

In [ ]:
from lenstronomy.Data.imaging_data import ImageData

pixel_scale = 0.06  # Adjust to your FITS files' actual pixel scale

# Set up image data for lenstronomy
data_class = ImageData(image_data=image_data, pixel_scale=pixel_scale)


5. Define Source Light Model
Use a simple model to start, and expand later if needed.

In [ ]:
from lenstronomy.LightModel.light_model import LightModel

source_model_list = ['SERSIC_ELLIPSE']
source_model = LightModel(light_model_list=source_model_list)

kwargs_source = [{
    'amp': 100,
    'R_sersic': 0.3,
    'n_sersic': 2.0,
    'e1': 0.0,
    'e2': 0.0,
    'center_x': 0.0,
    'center_y': 0.0
}]

6. Image Simulation and Visualization

In [ ]:
from lenstronomy.ImSim.image_model import ImageModel

imageModel = ImageModel(data_class=data_class,
                        lens_model_class=lens_model,
                        source_model_class=source_model)

# Generate model image
model_image = imageModel.image(kwargs_lens=kwargs_lens, kwargs_source=kwargs_source)

# Plot the modeled lensing effect
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('Observed Data')
plt.imshow(image_data, origin='lower', cmap='inferno')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.title('Modeled Lensing (SIE)')
plt.imshow(model_image, origin='lower', cmap='inferno')
plt.colorbar()

plt.tight_layout()
plt.show()


7. External Validation of 3T_E Gravity

In [ ]:
# Replace with your derived 3T_E parameters
kwargs_lens_3TE = [{
    'theta_E': 1.2,
    'e1': 0.05,
    'e2': 0.02,
    'center_x': 0.01,
    'center_y': -0.02
}]

# Generate 3T_E predicted lensing image
model_image_3TE = imageModel.image(kwargs_lens=kwargs_lens_3TE, kwargs_source=kwargs_source)

# Visualization
plt.figure(figsize=(18, 6))

plt.subplot(1, 3, 1)
plt.title('Observed Data')
plt.imshow(image_data, origin='lower', cmap='inferno')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.title('SIE (Reference)')
plt.imshow(model_image, origin='lower', cmap='inferno')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.title('3T_E Predicted')
plt.imshow(model_image_3TE, origin='lower', cmap='inferno')
plt.colorbar()

plt.tight_layout()
plt.show()


8. Statistical Comparison and Validation (Optional but recommended)

In [ ]:
from lenstronomy.Workflow.fitting_sequence import FittingSequence
from scipy.stats import linregress, kstest

# Setup for fitting sequence
kwargs_params = {
    'lens_model': [kwargs_lens_3TE],
    'source_model': [kwargs_source],
}

fitting_seq = FittingSequence(
    kwargs_data={'image_data': image_data, 'pixel_scale': pixel_scale},
    kwargs_model={
        'lens_model_list': lens_model_list,
        'source_light_model_list': source_model_list,
    },
    kwargs_constraints={},
    kwargs_likelihood={},
    kwargs_params=kwargs_params,
)

# Run Monte Carlo parameter fitting (MCMC)
fitting_kwargs_list = [['MCMC', {'n_burn': 200, 'n_run': 500, 'walkerRatio': 10}]]
chain_list = fitting_seq.fit_sequence(fitting_kwargs_list)

# Retrieve best-fit model
result = fitting_seq.best_fit()

# Statistical Bias Analysis
observed_flat = image_data.flatten()
predicted_flat = model_image_3TE.flatten()

# Clean NaN values explicitly
valid_mask = ~np.isnan(observed_flat) & ~np.isnan(predicted_flat)
observed_clean = observed_flat[valid_mask]
predicted_clean = predicted_flat[valid_mask]

# Linear regression
slope, intercept, r_value, p_value, std_err = linregress(predicted_clean, observed_clean)

# Standardized residuals for K-S test
residuals = observed_clean - predicted_clean
standardized_residuals = (residuals - residuals.mean()) / residuals.std()

# Kolmogorov-Smirnov test
ks_stat, ks_p = kstest(standardized_residuals, 'norm')

# Display statistical results
print("\nLinear Regression Results:")
print(f"Slope: {slope:.4f}, Intercept: {intercept:.4f}, R-value: {r_value:.4f}, p-value: {p_value:.4e}, Std Error: {std_err:.4f}")

print("\nKolmogorov-Smirnov Test Results:")
print(f"KS Statistic: {ks_stat:.4f}, KS p-value: {ks_p:.4f}")


Workspace - Parameter Search for Lenstronomy variables

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
# Navigate to working directory
import os
os.chdir('/content/drive/My Drive/AI/3TE/LENSING/')

# Confirm file is accessible
!ls

 hlsp_frontier_hst_acs-60mas_abell370_f435w_v1.0-epoch1_drz.fits
 hlsp_frontier_hst_acs-60mas_abell370_f475w_v0.2_drz.fits
 hlsp_frontier_hst_acs-60mas_abell370_f814w_v1.0-epoch1_drz.fits
 hlsp_frontier_hst_acs-60mas_abell370-hffpar_f814w_v1.0-epoch1_drz.fits
 hlsp_frontier_hst_acs-60mas_abell370-hffpar_f814w_v1.0-epoch2_drz.fits
 hlsp_frontier_hst_acs-60mas-selfcal_abell370-hffpar_f435w_v1.0-epoch2_drz.fits
 hlsp_frontier_hst_wfc3-60mas_abell370_f160w_v1.0-epoch2_drz.fits
 hlsp_frontier_hst_wfc3-60mas-bkgdcor_abell370_f125w_v1.0-epoch2_drz.fits
 hlsp_frontier_hst_wfc3-60mas-bkgdcor_abell370_f160w_v1.0-epoch2_drz.fits
 hlsp_frontier_hst_wfc3ir-60mas_abell370_f140w_v0.2_drz.fits
 hlsp_frontier_hst_wfc3ir-60mas_abell370_f160w_v0.2_drz.fits
'LENSING assumptions.gdoc'
 parameter_search_summary.csv
 SOURCES:.gdoc


In [12]:
import pandas as pd

# Load parameter search results
results_df = pd.read_csv('parameter_search_summary.csv')

# Inspect the DataFrame structure and content
print(results_df.head())


                                                File  Best_alpha  \
0  hlsp_frontier_hst_acs-60mas-selfcal_abell370-h...      0.0001   
1  hlsp_frontier_hst_acs-60mas_abell370-hffpar_f8...      0.0001   
2  hlsp_frontier_hst_acs-60mas_abell370-hffpar_f8...      0.0001   
3  hlsp_frontier_hst_acs-60mas_abell370_f435w_v1....      0.0001   
4  hlsp_frontier_hst_acs-60mas_abell370_f475w_v0....      0.0001   

   Best_entropy_gradient  Mean_Potential  CI_Lower  CI_Upper  LR_slope  \
0               0.000100        0.000074 -0.000084 -0.000011       1.0   
1               0.007525        0.001067  0.000566  0.000867       1.0   
2               0.000100        0.001032  0.000475  0.000740       1.0   
3               0.005050        0.001168  0.000201  0.000289       1.0   
4               0.000100        0.001640  0.000365  0.001049       1.0   

   LR_intercept  LR_r_value  LR_p_value   KS_stat  KS_p  
0  3.442613e-15         1.0         0.0  0.472087   0.0  
1 -1.147252e-10         1.0   

In [14]:
# Load parameter search results
results_df = pd.read_csv('parameter_search_summary.csv')

# Identify optimal parameters (row with highest Mean_Potential)
optimal_row = results_df.loc[results_df['Mean_Potential'].idxmax()]
print("Optimal Parameters Found:")
print(optimal_row)



Optimal Parameters Found:
File                     hlsp_frontier_hst_wfc3ir-60mas_abell370_f140w_...
Best_alpha                                                          0.0001
Best_entropy_gradient                                               0.0001
Mean_Potential                                                    0.008894
CI_Lower                                                          0.005262
CI_Upper                                                          0.020388
LR_slope                                                               1.0
LR_intercept                                                          -0.0
LR_r_value                                                             1.0
LR_p_value                                                             0.0
KS_stat                                                           0.498072
KS_p                                                                   0.0
Name: 9, dtype: object
